# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [2]:
!pip install -q transformers 
!pip install -q peft
!pip install -q evaluate
!pip install -q scikit-learn

In [3]:
from peft import LoraConfig, TaskType
import os

from transformers import BertForSequenceClassification
from peft import get_peft_model
# Import the datasets and transformers packages
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
import evaluate


config = LoraConfig(task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1)

In [4]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased', 
    num_labels=2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
lora_model = get_peft_model(model, config)

In [6]:
lora_model.print_trainable_parameters()


trainable params: 39,940 || all params: 108,350,212 || trainable%: 0.03686194910260074


In [7]:
# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("imdb", split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(1000))

# Show the dataset
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 1000
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 1000
 })}

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def preprocess_function(examples):
    """Preprocess the imdb dataset by returning tokenized examples."""
    tokens = tokenizer(examples['text'], padding="max_length", truncation=True)
    return tokens


tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)


In [9]:
tokenized_ds['train'] = tokenized_ds['train'].select_columns(['input_ids', 'label']).rename_column("label", "labels")
tokenized_ds['test'] = tokenized_ds['test'].select_columns(['input_ids', 'label']).rename_column("label", "labels")

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(eval_pred):
    logits = eval_pred.predictions
    labels = eval_pred.label_ids
    predictions = np.argmax(logits, axis=-1)
    # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, predictions)
    precision = precision_score(labels, predictions)
    recall = recall_score(labels, predictions)
    accuracy = accuracy_score(labels, predictions)
    
    return {'f1': f1,
           'precision_score':precision,
           'recall_score':recall,
           'accuracy_score':accuracy}

In [11]:
batch_size = 5
args = TrainingArguments(
        output_dir='test_trainer',
        remove_unused_columns=False,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=10e-3,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=batch_size,
        fp16=True,
        num_train_epochs=1,
        logging_steps=10,
        load_best_model_at_end=True,
        label_names=["labels"]
    )
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.7286738157272339,
 'eval_f1': 0.5746569814366425,
 'eval_precision_score': 0.474034620505992,
 'eval_recall_score': 0.7295081967213115,
 'eval_accuracy_score': 0.473,
 'eval_runtime': 14.2243,
 'eval_samples_per_second': 70.302,
 'eval_steps_per_second': 14.06}

In [12]:
trainer.save_model("benchmark")

In [13]:
batch_size = 5
args = TrainingArguments(
        output_dir='test_trainer_lora',
        remove_unused_columns=False,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=10e-3,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=batch_size,
        fp16=True,
        num_train_epochs=1,
        logging_steps=10,
        load_best_model_at_end=True,
        label_names=["labels"]
    )
trainer = Trainer(
    model=lora_model,
    args=args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()
trainer.save_model("lora_model")

Epoch,Training Loss,Validation Loss,F1,Precision Score,Recall Score,Accuracy Score
1,0.835600,0.762113,0.118081,0.592593,0.065574,0.522000


Checkpoint destination directory test_trainer_lora/checkpoint-50 already exists and is non-empty.Saving will proceed but saved results may be invalid.


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [14]:
from transformers import BertForSequenceClassification
lora_model = BertForSequenceClassification.from_pretrained("lora_model")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
trainer = Trainer(
    model=lora_model,
    args=args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Show the performance of the model on the test set
# What do you think the evaluation accuracy will be?
trainer.evaluate()

{'eval_loss': 0.762112557888031,
 'eval_f1': 0.11808118081180811,
 'eval_precision_score': 0.5925925925925926,
 'eval_recall_score': 0.06557377049180328,
 'eval_accuracy_score': 0.522,
 'eval_runtime': 13.5835,
 'eval_samples_per_second': 73.619,
 'eval_steps_per_second': 14.724}

In [16]:
lora_model.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(
                in_features=768, out_features=768, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=1, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=1, out_features=768, bias=Fa

In [17]:
inputs = tokenizer.encode("This is very nice movie, but horrible", return_tensors="pt").to('cuda')
outputs = lora_model(input_ids=inputs)
predictions = outputs.logits.argmax(dim=1)[0]

if predictions==1:
    print("Positive")
else:
    print("Negative")

Negative
